# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION= {'class': 'SimpleStrategy', 'replication_factor': 1}")

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:

### Here will create the table containing the artist's name, the song, it's length, the item in session and the id of the session with session and user IDs as primary keys to make give each row a unique value ID.

### Then we insert our data into the table 

### And finally we test it out by extracting the data for the session id of 338 while the item in session is 4 to test it out.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_length_session(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
 
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_length_session(sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = " SELECT artist, song, length FROM song_length_session WHERE sessionId=338 AND itemInSession=4 "

rows = session.execute(query)

for row in rows:
        print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2:

### Here will create the table containing the artist's name, the song, the session id, the user's id and the user's first and last name with primary keys as the session id and user id to again avoid the removal of duplicated value rows and with the item in session as a clustering column to sort the data by the item in session.

### Then we insert our data into the table

### And finally we test it out by extracting the data for the user with with id number 10 and while the session number is 182 while viewing only the user's and the artist's name with the name of the song.

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_2 = "CREATE TABLE IF NOT EXISTS artist_song_with_users"
query_2 = query_2 + "(sessionId int, userId int, itemInSession int, song text, artist text, firstName text, lastName text, PRIMARY KEY((sessionId, userId), itemInSession))" 

try:
    session.execute(query_2)
except Exception as e:
    print(e)
                   

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assign the INSERT statements into the `query` variable
        query_3 = "INSERT INTO artist_song_with_users(sessionId, userId, itemInSession, song, artist, firstName, lastName)"
        query_3 = query_3 + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        
        session.execute(query_3, (int(line[8]), int(line[10]), int(line[8]), line[9], line[0], line[1], line[4]))
                

In [13]:
query_y = "SELECT artist, song, firstName, lastName FROM artist_song_with_users WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query_y)
except Exception as e:
    print(e)

for row in rows:
        print(row.artist, row.song, row.firstname, row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3:

### Here will create the table containing thet the user id , the user's first , last name and the name of the song played during each session with the primary keys as the user's id number and the name of the song.

### Then we insert our data into the table

### And finally we test it out by extracting the data for all the user who have listen to the song 'All Hands Against His Own' during this session .

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_4 = "CREATE TABLE IF NOT EXISTS user_name_with_songs(userId int, song text, firstName text, lastName text, PRIMARY KEY(userId, song))"  

try:
    session.execute(query_4)
except Exception as e:
    print(e)

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query_5 = "INSERT INTO user_name_with_songs(userId, song, firstName, lastName)"
        query_5 = query_5 + "VALUES (%s,%s,%s,%s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query_5, (int(line[10]), line[9], line[1], line[4]))                   

In [16]:
query_z = "SELECT firstName, lastName FROM user_name_with_songs WHERE song= 'All Hands Against His Own' ALLOW FILTERING"
    
rows = session.execute(query_z)

for row in rows:
        print(row.firstname, row.lastname)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE song_length_session")
session.execute("DROP TABLE artist_song_with_users")
session.execute("DROP TABLE user_name_with_songs")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()

refrences:
https://blog.katastros.com/a?ID=00750-e0818c8a-ea4f-4513-9ec2-260116b3e1dd
https://cassandra.apache.org/doc/latest/cassandra/cql/index.html